Referensi
- https://colab.research.google.com/github/ShawonAshraf/nlu-jointbert-dl2021/blob/main/notebooks/nlu_jointbert_dl21.ipynb

Note: sesuaikan semua `/path/to/` menuju directory yang sesuai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installs

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 13.7 MB/s 
     |████████████████████████████████| 596 kB 70.8 MB/s 
     |████████████████████████████████| 6.6 MB 42.8 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Imports

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline
figsize = (15, 10)

In [ ]:
df_atis_test = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/atis/test/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])
df_atis_train = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/atis/train/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])
df_atis_validation = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/atis/valid/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])

df_snips_test = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/snips/test/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])
df_snips_train = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/snips/train/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])
df_snips_validation = pd.read_csv('https://raw.githubusercontent.com/moore3930/SlotRefine/main/data/snips/valid/data', header=None, sep="\t", names=["data", "encoding", "ground_truth"])

In [ ]:
def create_O_slot(text):
  temp = "O"
  for i in range(1,len(text.split(" "))):
    temp += " O"
  return temp

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

# Tokenize

In [ ]:
texts = [d for d in df_snips_train.data]
slots = [d for d in df_snips_train.encoding]
all_O = []
for i in slots:
  all_O.append(create_O_slot(i))
all_O

['O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O O O O O O O O O',
 'O O O O O O',
 'O O O O O O O O O O',
 'O O O O O O O O O O',
 'O O O O O O O',
 'O O O O O O O',
 'O O O',
 'O O O O O O O',
 'O O O O O O O O O O O O O O O',
 'O O O O O',
 'O O O O O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O',
 'O O O O O O O',
 'O O O O O O O O',
 'O O O O',
 'O O O O O',
 'O O O O O O O O O',
 'O O O',
 'O O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O',
 'O O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O O O O O',
 'O O O O O O O O O O O O',
 'O O O O O O',
 'O O O O O O O O O',
 'O O O O O O O',
 'O O O O O',
 'O O O O O O O O O O O',
 'O O O O O O O O O O',
 'O O O O O O',
 'O O O O O O O O O O O O O O O',
 'O O O O O O O O O O O',
 'O O O 

In [ ]:
def encode_texts(tokenizer, texts, slots, max_length = None, padding = True):
    return tokenizer(texts,slots, padding= padding,truncation=True, max_length = max_length, return_tensors="tf")
max_token_length = encode_texts(tokenizer, texts, slots)["input_ids"].shape[1]
tds = encode_texts(tokenizer, texts, all_O, max_token_length, "max_length")
tds.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
{"input_ids": tds["input_ids"], "token_type_ids": tds["token_type_ids"],  "attention_mask": tds["attention_mask"]}

{'attention_mask': <tf.Tensor: shape=(13084, 145), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(13084, 145), dtype=int32, numpy=
 array([[ 101, 5113, 1106, ...,    0,    0,    0],
        [ 101, 5194, 2585, ...,    0,    0,    0],
        [ 101,  178, 1660, ...,    0,    0,    0],
        ...,
        [ 101, 2603, 1103, ...,    0,    0,    0],
        [ 101, 1301, 1106, ...,    0,    0,    0],
        [ 101, 2603, 3987, ...,    0,    0,    0]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(13084, 145), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
encoded_texts = tds

# Mapping slots and intents

In [ ]:
intents = [d for d in df_snips_train.ground_truth]
intent_names = list(set(intents))
intent_names.sort()

In [ ]:
intent_map = dict() # index -> intent
for idx, ui in enumerate(intent_names):
    intent_map[ui] = idx
intent_map

{'AddToPlaylist': 0,
 'BookRestaurant': 1,
 'GetWeather': 2,
 'PlayMusic': 3,
 'RateBook': 4,
 'SearchCreativeWork': 5,
 'SearchScreeningEvent': 6}

In [ ]:
# map to train_data values
def encode_intents(intents, intent_map):
    encoded = []
    for i in intents:
        encoded.append(intent_map[i])
    # convert to tf tensor
    return tf.convert_to_tensor(encoded, dtype="int32")

encoded_intents = encode_intents(intents, intent_map)

In [ ]:
# encode slots
slot_list = [x.split(" ") for x in df_snips_train.encoding]
slot_names = set()
for td in slot_list:
    for slot in td:
        slot_names.add(slot)
slot_names = list(slot_names)
slot_names.sort()
slot_names.insert(0, "<PAD>")

In [ ]:
slot_map = dict() # slot -> index
for idx, us in enumerate(slot_names):
    slot_map[us] = idx

In [ ]:
# gets slot name from its values
def get_slot_from_word(word, text, slot):
    text = text.split()
    for i in range(len(text)):
        if word == text[i]:
            return slot[i]
    return None

# print(df_atis_train.data[0])
# print(df_atis_train.encoding[0])
# print("slot_name for google is : ", get_slot_from_word("google", df_snips_train.data[0], slot_list[0]))

In [ ]:
import numpy as np

# find the max encoded test length
# tokenizer pads all texts to same length anyway so
# just get the length of the first one's input_ids
max_len = len(encoded_texts["input_ids"][0])

def encode_slots(all_slots, all_texts, 
                 tokenizer, slot_map, max_len=max_len):
    encoded_slots = np.zeros(shape=(len(all_texts), max_len), dtype=np.int32)
    
    for idx, text in enumerate(all_texts):
        enc = [] # for this idx, to be added at the end to encoded_slots
        
        # slot names for this idx
        slot_names = all_slots[idx]
        
        # raw word tokens
        # not using bert for this block because bert uses
        # a wordpiece tokenizer which will make 
        # the slot label to word mapping
        # difficult
        raw_tokens = text.split()

        # words or slot_values associated with a certain
        # slot_name are contained in the values of the
        # dict slots_names
        # now this becomes a two way lookup
        # first we check if a word belongs to any
        # slot label or not and then we add the value from
        # slot map to encoded for that word
        for rt in raw_tokens:
            # use bert tokenizer
            # to get wordpiece tokens
            bert_tokens = tokenizer.tokenize(rt)
            
            # find the slot name for a token
            rt_slot_name = get_slot_from_word(rt, text, slot_names)
            if rt_slot_name is not None:
                # fill with the slot_map value for all ber tokens for rt
                enc.append(slot_map[rt_slot_name])
                enc.extend([slot_map[rt_slot_name]] * (len(bert_tokens) - 1))

            else:
                # rt is not associated with any slot name
                enc.append(0)

        
        # now add to encoded_slots
        # ignore the first and the last elements
        # in encoded text as they're special chars
        encoded_slots[idx, 1:len(enc)+1] = enc
    
    return encoded_slots
    

In [ ]:
encoded_slots = encode_slots(slot_list, texts, tokenizer, slot_map)

In [ ]:
encoded_slots[0]

array([ 0, 72, 72,  2,  2,  2, 72, 72, 72,  1,  1, 72, 33, 33, 33, 66,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

# Model

In [ ]:
from transformers import TFBertModel
from keras.layers import Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy

class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, slot_num_labels=None,
                 model_name=model_name, dropout_prob=0.1):
        super().__init__(name="joint_intent_slot")
        self.bert = TFBertModel.from_pretrained(model_name)
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(intent_num_labels,
                                       name="intent_classifier")
        self.slot_classifier = Dense(slot_num_labels,
                                     name="slot_classifier")
    def call(self, inputs, **kwargs):
        # two outputs from BERT
        trained_bert = self.bert.bert(inputs, **kwargs)
        pooled_output = trained_bert.pooler_output
        sequence_output = trained_bert.last_hidden_state
        
        # sequence_output will be used for slot_filling / classification
        sequence_output = self.dropout(sequence_output,
                                       training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(sequence_output)

        # pooled_output for intent classification
        pooled_output = self.dropout(pooled_output,
                                     training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(pooled_output)

        return slot_logits, intent_logits


In [ ]:
joint_model = JointIntentAndSlotFillingModel(
    intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
opt = Adam(learning_rate=3e-5, epsilon=1e-08)

# two outputs, one for slots, another for intents
# we have to fine tune for both
losses = [SparseCategoricalCrossentropy(from_logits=True),
          SparseCategoricalCrossentropy(from_logits=True)]

metrics = [SparseCategoricalAccuracy("accuracy")]
# compile model
joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)

In [ ]:
# x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}

# joint_model.load_weights('/content/drive/MyDrive/ML/TK/second_pass_epoch_1_weight/')

In [ ]:
joint_model.summary()

Model: "joint_intent_slot"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_bert_model (TFBertModel)  multiple                 108310272 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 intent_classifier (Dense)   multiple                  5383      
                                                                 
 slot_classifier (Dense)     multiple                  56137     
                                                                 
Total params: 108,371,792
Trainable params: 108,371,792
Non-trainable params: 0
_________________________________________________________________


# First Pass

In [ ]:
x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}

history = joint_model.fit(
    x, (encoded_slots, encoded_intents), epochs=1, batch_size=32, shuffle=True)

409/409 [==============================] - 426s 1s/step - loss: 0.0306 - output_1_loss: 0.0157 - output_2_loss: 0.0149 - output_1_accuracy: 0.9953 - output_2_accuracy: 0.9960


In [ ]:
def nlu(text, tokenizer, model, intent_names, slot_names):
    # inputs = tf.constant(tokenizer.encode(text, create_O_slot(text)))[None, :]  # batch_size = 1
    # inputs = tf.constant(tokenizer.encode(text))[None, :]
    encoded_texts = tokenizer(text, padding= "max_length",truncation=True, max_length = max_token_length, return_tensors="tf")
    x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}
    inputs = x
    outputs = model(inputs)
    slot_logits, intent_logits = outputs

    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, :]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    info = {"intent": intent_names[intent_id], "slots": {}}
    out_dict = {}
    # get all slot names and add to out_dict as keys
    predicted_slots = set([slot_names[s] for s in slot_ids if s != 0])
    for ps in predicted_slots:
      out_dict[ps] = []

    # check if the text starts with a small letter
    if text[0].islower():
      tokens = tokenizer.tokenize(text, add_special_tokens=True)
    else:
      tokens = tokenizer.tokenize(text)
    slot_list = []
    for token, slot_id in zip(tokens, slot_ids):
        slot_name = slot_names[slot_id]
        if slot_name == "<PAD>":
            continue
        if token == "[CLS]":
            continue
        if token == "[SEP]":
            continue

        # collect tokens
        collected_tokens = [token]
        idx = tokens.index(token)

        # see if it starts with ##
        # then it belongs to the previous token
        if token.startswith("##"):
          # check if the token already exists or not
          if tokens[idx - 1] not in out_dict[slot_name]:
            collected_tokens.insert(0, tokens[idx - 1])
        else:
          slot_list.append(slot_name)
        # add collected tokens to slots
        out_dict[slot_name].extend(collected_tokens)

    # process out_dict
    for slot_name in out_dict:
        tokens = out_dict[slot_name]
        slot_value = tokenizer.convert_tokens_to_string(tokens)

        info["slots"][slot_name] = slot_value.strip()
    info["slot list"] = slot_list

    return info


Sesuaikan path di bawah sesuai kebutuhan.

In [ ]:
joint_model.save("path/to/Joint Bert/Snips_2/first_pass_epoch_X_weight")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Manusia Learning/Joint Bert/Snips_2/first_pass_epoch_3_weight/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Manusia Learning/Joint Bert/Snips_2/first_pass_epoch_3_weight/assets


In [ ]:
df_snips_test.data[12]

'find on dress parade'

In [ ]:
df_snips_test.encoding[12]

'O B-movie_name I-movie_name I-movie_name'

In [ ]:
nlu(df_snips_test.data[12], tokenizer, joint_model, intent_names, slot_names)

{'intent': 'SearchCreativeWork',
 'slot list': ['O', 'B-object_name', 'B-object_name', 'I-object_name'],
 'slots': {'B-object_name': 'on dress',
  'I-object_name': 'parade',
  'O': 'find'}}

# Evaluate First Pass

In [ ]:
pred_slot_first_pass = []
pred_intent_first_pass = []
for i in df_snips_test.values:
  info = nlu(i[0], tokenizer, joint_model, intent_names, slot_names)
  pred_slot_first_pass.append(info["slot list"])
  pred_intent_first_pass.append(info["intent"])

In [ ]:
test_slot = [i.split(" ") for i in df_snips_test.encoding]

In [ ]:
test_intent = df_snips_test.ground_truth.values

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(test_slot)
test_bin = mlb.transform(test_slot)
pred_bin = mlb.transform(pred_slot_first_pass)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_bin, pred_bin, target_names=mlb.classes_))

                              precision    recall  f1-score   support

                     B-album       0.67      0.40      0.50        10
                    B-artist       0.92      0.96      0.94       107
               B-best_rating       1.00      1.00      1.00        43
                      B-city       0.77      0.85      0.81        60
     B-condition_description       0.97      1.00      0.98        28
     B-condition_temperature       1.00      1.00      1.00        23
                   B-country       0.76      0.70      0.73        44
                   B-cuisine       0.75      0.64      0.69        14
          B-current_location       1.00      1.00      1.00        14
               B-entity_name       0.86      0.76      0.81        33
                  B-facility       1.00      1.00      1.00         3
                     B-genre       1.00      0.80      0.89         5
            B-geographic_poi       1.00      1.00      1.00        11
             B-loca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_intent, pred_intent_first_pass))

                      precision    recall  f1-score   support

       AddToPlaylist       1.00      1.00      1.00       124
      BookRestaurant       0.96      1.00      0.98        92
          GetWeather       1.00      0.98      0.99       104
           PlayMusic       0.99      0.94      0.96        86
            RateBook       1.00      1.00      1.00        80
  SearchCreativeWork       0.87      0.97      0.92       107
SearchScreeningEvent       1.00      0.91      0.95       107

            accuracy                           0.97       700
           macro avg       0.97      0.97      0.97       700
        weighted avg       0.97      0.97      0.97       700



In [ ]:
!pip install nervaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
new_slot_names = set()
for i in slot_names:
  if (i[0] == "B" or i[0] == "I"):
    new_slot_names.add(i[2:])
  else:
    new_slot_names.add(i)
# new_slot_names

In [ ]:
from nervaluate import Evaluator

res, res_type = Evaluator(test_slot, pred_slot_first_pass, tags=new_slot_names, loader="list").evaluate()

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,1643.000000,1583.000000,1513.000000,1583.000000
incorrect,120.000000,0.000000,250.000000,180.000000
partial,0.000000,180.000000,0.000000,0.000000
missed,27.000000,27.000000,27.000000,27.000000
spurious,166.000000,166.000000,166.000000,166.000000
possible,1790.000000,1790.000000,1790.000000,1790.000000
actual,1929.000000,1929.000000,1929.000000,1929.000000
precision,0.851737,0.867289,0.784344,0.820632
recall,0.917877,0.934637,0.845251,0.884358
f1,0.883571,0.899704,0.813660,0.851304


# Create B-only Slot

In [ ]:
first_slot = []
for i in df_snips_train.values:
  info = nlu(i[0], tokenizer, joint_model, intent_names, slot_names)
  first_slot.append(info["slot list"])

In [ ]:
def only_B_slot(text):
  temp = ""
  for i in text:
    if (i[0] == 'B'):
      temp += (" " + i)
    else:
      temp += " O"
  return temp.strip()

In [ ]:
second_slot = []
for i in first_slot:
  second_slot.append(only_B_slot(i))

In [ ]:
tds = encode_texts(tokenizer, texts, second_slot, max_token_length, "max_length")
tds.keys()
encoded_texts = tds

In [ ]:
{"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}

{'attention_mask': <tf.Tensor: shape=(13084, 145), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(13084, 145), dtype=int32, numpy=
 array([[ 101, 5113, 1106, ...,    0,    0,    0],
        [ 101, 5194, 2585, ...,    0,    0,    0],
        [ 101,  178, 1660, ...,    0,    0,    0],
        ...,
        [ 101, 2603, 1103, ...,    0,    0,    0],
        [ 101, 1301, 1106, ...,    0,    0,    0],
        [ 101, 2603, 3987, ...,    0,    0,    0]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(13084, 145), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>}

# Second Pass


In [ ]:
x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}

history = joint_model.fit(
    x, (encoded_slots, encoded_intents), epochs=1, batch_size=32, shuffle=True)

409/409 [==============================] - 414s 1s/step - loss: 0.0157 - output_1_loss: 0.0120 - output_2_loss: 0.0038 - output_1_accuracy: 0.9963 - output_2_accuracy: 0.9990


In [ ]:
df_snips_test.data[123]

'i would rate that old ace in the hole one stars and a best rating of 6'

In [ ]:
df_snips_test.encoding[123]

'O O O B-object_name I-object_name I-object_name I-object_name I-object_name I-object_name B-rating_value B-rating_unit O O O O O B-best_rating'

In [ ]:
nlu(df_snips_test.data[123], tokenizer, joint_model, intent_names, slot_names)

{'intent': 'RateBook',
 'slot list': ['O',
  'O',
  'O',
  'B-object_name',
  'I-object_name',
  'I-object_name',
  'I-object_name',
  'I-object_name',
  'I-object_name',
  'B-rating_value',
  'B-rating_unit',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-best_rating'],
 'slots': {'B-best_rating': '6',
  'B-object_name': 'that',
  'B-rating_unit': 'stars',
  'B-rating_value': 'one',
  'I-object_name': 'old ace in the hole',
  'O': 'i would rate and a best rating of'}}

In [ ]:
pred_slot_second_pass = []
pred_intent_second_pass = []
for i in df_snips_test.values:
  info = nlu(i[0], tokenizer, joint_model, intent_names, slot_names)
  pred_slot_second_pass.append(info["slot list"])
  pred_intent_second_pass.append(info["intent"])

# Evaluate Second Pass

In [ ]:
test_slot = [i.split(" ") for i in df_snips_test.encoding]

In [ ]:
test_intent = df_snips_test.ground_truth.values

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(test_slot)
test_bin = mlb.transform(test_slot)
pred_bin = mlb.transform(pred_slot_second_pass)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_bin, pred_bin, target_names=mlb.classes_))

                              precision    recall  f1-score   support

                     B-album       0.56      0.50      0.53        10
                    B-artist       0.96      0.92      0.94       107
               B-best_rating       1.00      1.00      1.00        43
                      B-city       0.89      0.83      0.86        60
     B-condition_description       1.00      1.00      1.00        28
     B-condition_temperature       1.00      1.00      1.00        23
                   B-country       0.81      0.89      0.85        44
                   B-cuisine       0.81      0.93      0.87        14
          B-current_location       1.00      1.00      1.00        14
               B-entity_name       0.75      0.91      0.82        33
                  B-facility       1.00      1.00      1.00         3
                     B-genre       1.00      0.80      0.89         5
            B-geographic_poi       1.00      1.00      1.00        11
             B-loca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_intent, pred_intent_second_pass))

                      precision    recall  f1-score   support

       AddToPlaylist       1.00      1.00      1.00       124
      BookRestaurant       0.97      1.00      0.98        92
          GetWeather       1.00      0.98      0.99       104
           PlayMusic       0.96      0.99      0.97        86
            RateBook       1.00      1.00      1.00        80
  SearchCreativeWork       0.93      0.94      0.94       107
SearchScreeningEvent       0.99      0.93      0.96       107

            accuracy                           0.98       700
           macro avg       0.98      0.98      0.98       700
        weighted avg       0.98      0.98      0.98       700



Sesuaikan path ini sesuai kebutuhan

In [ ]:
joint_model.save("path/to/Joint Bert/Snips_2/second_pass_epoch_X_weight")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Manusia Learning/Joint Bert/Snips_2/second_pass_epoch_3_weight/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Manusia Learning/Joint Bert/Snips_2/second_pass_epoch_3_weight/assets


In [ ]:
from nervaluate import Evaluator

res, res_type = Evaluator(test_slot, pred_slot_second_pass, tags=new_slot_names, loader="list").evaluate()

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,1658.000000,1600.000000,1541.000000,1600.000000
incorrect,103.000000,0.000000,220.000000,161.000000
partial,0.000000,161.000000,0.000000,0.000000
missed,29.000000,29.000000,29.000000,29.000000
spurious,163.000000,163.000000,163.000000,163.000000
possible,1790.000000,1790.000000,1790.000000,1790.000000
actual,1924.000000,1924.000000,1924.000000,1924.000000
precision,0.861746,0.873441,0.800936,0.831601
recall,0.926257,0.938827,0.860894,0.893855
f1,0.892838,0.904954,0.829833,0.861605


# Ringkasan evaluasi setiap part dan epoch

## SNIPS

First pass epoch 1

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,1471.000000,1308.000000,1161.000000,1308.000000
incorrect,239.000000,0.000000,549.000000,402.000000
partial,0.000000,402.000000,0.000000,0.000000
missed,80.000000,80.000000,80.000000,80.000000
spurious,411.000000,411.000000,411.000000,411.000000
possible,1790.000000,1790.000000,1790.000000,1790.000000
actual,2121.000000,2121.000000,2121.000000,2121.000000
precision,0.693541,0.711457,0.547383,0.616690
recall,0.821788,0.843017,0.648603,0.730726
f1,0.752237,0.771670,0.593710,0.668883


Second pass epoch 1

In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,1575.000000,1459.000000,1356.000000,1459.000000
incorrect,176.000000,0.000000,395.000000,292.000000
partial,0.000000,292.000000,0.000000,0.000000
missed,39.000000,39.000000,39.000000,39.000000
spurious,277.000000,277.000000,277.000000,277.000000
possible,1790.000000,1790.000000,1790.000000,1790.000000
actual,2028.000000,2028.000000,2028.000000,2028.000000
precision,0.776627,0.791420,0.668639,0.719428
recall,0.879888,0.896648,0.757542,0.815084
f1,0.825039,0.840754,0.710320,0.764274


First pass epoch 2

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_bin, pred_bin, target_names=mlb.classes_))

                              precision    recall  f1-score   support

                     B-album       0.00      0.00      0.00        10
                    B-artist       0.95      0.81      0.87       107
               B-best_rating       1.00      1.00      1.00        43
                      B-city       0.77      0.90      0.83        60
     B-condition_description       0.97      1.00      0.98        28
     B-condition_temperature       0.96      1.00      0.98        23
                   B-country       0.85      0.64      0.73        44
                   B-cuisine       0.73      0.79      0.76        14
          B-current_location       1.00      0.93      0.96        14
               B-entity_name       0.67      0.85      0.75        33
                  B-facility       1.00      1.00      1.00         3
                     B-genre       0.80      0.80      0.80         5
            B-geographic_poi       1.00      1.00      1.00        11
             B-loca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_intent, pred_intent_first_pass))

                      precision    recall  f1-score   support

       AddToPlaylist       1.00      1.00      1.00       124
      BookRestaurant       0.99      1.00      0.99        92
          GetWeather       1.00      0.99      1.00       104
           PlayMusic       0.98      1.00      0.99        86
            RateBook       1.00      1.00      1.00        80
  SearchCreativeWork       0.92      0.97      0.95       107
SearchScreeningEvent       0.99      0.92      0.95       107

            accuracy                           0.98       700
           macro avg       0.98      0.98      0.98       700
        weighted avg       0.98      0.98      0.98       700



In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,1621.000000,1549.000000,1471.000000,1549.000000
incorrect,142.000000,0.000000,292.000000,214.000000
partial,0.000000,214.000000,0.000000,0.000000
missed,27.000000,27.000000,27.000000,27.000000
spurious,217.000000,217.000000,217.000000,217.000000
possible,1790.000000,1790.000000,1790.000000,1790.000000
actual,1980.000000,1980.000000,1980.000000,1980.000000
precision,0.818687,0.836364,0.742929,0.782323
recall,0.905587,0.925140,0.821788,0.865363
f1,0.859947,0.878515,0.780371,0.821751


Second pass epoch 2

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_bin, pred_bin, target_names=mlb.classes_))

                              precision    recall  f1-score   support

                     B-album       0.00      0.00      0.00        10
                    B-artist       0.95      0.81      0.87       107
               B-best_rating       1.00      1.00      1.00        43
                      B-city       0.77      0.90      0.83        60
     B-condition_description       0.97      1.00      0.98        28
     B-condition_temperature       0.96      1.00      0.98        23
                   B-country       0.85      0.64      0.73        44
                   B-cuisine       0.73      0.79      0.76        14
          B-current_location       1.00      0.93      0.96        14
               B-entity_name       0.67      0.85      0.75        33
                  B-facility       1.00      1.00      1.00         3
                     B-genre       0.80      0.80      0.80         5
            B-geographic_poi       1.00      1.00      1.00        11
             B-loca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_intent, pred_intent_first_pass))

                      precision    recall  f1-score   support

       AddToPlaylist       1.00      1.00      1.00       124
      BookRestaurant       0.99      1.00      0.99        92
          GetWeather       1.00      0.99      1.00       104
           PlayMusic       0.98      1.00      0.99        86
            RateBook       1.00      1.00      1.00        80
  SearchCreativeWork       0.92      0.97      0.95       107
SearchScreeningEvent       0.99      0.92      0.95       107

            accuracy                           0.98       700
           macro avg       0.98      0.98      0.98       700
        weighted avg       0.98      0.98      0.98       700



In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,1621.000000,1549.000000,1471.000000,1549.000000
incorrect,142.000000,0.000000,292.000000,214.000000
partial,0.000000,214.000000,0.000000,0.000000
missed,27.000000,27.000000,27.000000,27.000000
spurious,217.000000,217.000000,217.000000,217.000000
possible,1790.000000,1790.000000,1790.000000,1790.000000
actual,1980.000000,1980.000000,1980.000000,1980.000000
precision,0.818687,0.836364,0.742929,0.782323
recall,0.905587,0.925140,0.821788,0.865363
f1,0.859947,0.878515,0.780371,0.821751


First pass epoch 3

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_bin, pred_bin, target_names=mlb.classes_))

                              precision    recall  f1-score   support

                     B-album       0.67      0.40      0.50        10
                    B-artist       0.92      0.96      0.94       107
               B-best_rating       1.00      1.00      1.00        43
                      B-city       0.77      0.85      0.81        60
     B-condition_description       0.97      1.00      0.98        28
     B-condition_temperature       1.00      1.00      1.00        23
                   B-country       0.76      0.70      0.73        44
                   B-cuisine       0.75      0.64      0.69        14
          B-current_location       1.00      1.00      1.00        14
               B-entity_name       0.86      0.76      0.81        33
                  B-facility       1.00      1.00      1.00         3
                     B-genre       1.00      0.80      0.89         5
            B-geographic_poi       1.00      1.00      1.00        11
             B-loca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,1643.000000,1583.000000,1513.000000,1583.000000
incorrect,120.000000,0.000000,250.000000,180.000000
partial,0.000000,180.000000,0.000000,0.000000
missed,27.000000,27.000000,27.000000,27.000000
spurious,166.000000,166.000000,166.000000,166.000000
possible,1790.000000,1790.000000,1790.000000,1790.000000
actual,1929.000000,1929.000000,1929.000000,1929.000000
precision,0.851737,0.867289,0.784344,0.820632
recall,0.917877,0.934637,0.845251,0.884358
f1,0.883571,0.899704,0.813660,0.851304


Second pass epoch 3

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_bin, pred_bin, target_names=mlb.classes_))

                              precision    recall  f1-score   support

                     B-album       0.56      0.50      0.53        10
                    B-artist       0.96      0.92      0.94       107
               B-best_rating       1.00      1.00      1.00        43
                      B-city       0.89      0.83      0.86        60
     B-condition_description       1.00      1.00      1.00        28
     B-condition_temperature       1.00      1.00      1.00        23
                   B-country       0.81      0.89      0.85        44
                   B-cuisine       0.81      0.93      0.87        14
          B-current_location       1.00      1.00      1.00        14
               B-entity_name       0.75      0.91      0.82        33
                  B-facility       1.00      1.00      1.00         3
                     B-genre       1.00      0.80      0.89         5
            B-geographic_poi       1.00      1.00      1.00        11
             B-loca

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_intent, pred_intent_second_pass))

                      precision    recall  f1-score   support

       AddToPlaylist       1.00      1.00      1.00       124
      BookRestaurant       0.97      1.00      0.98        92
          GetWeather       1.00      0.98      0.99       104
           PlayMusic       0.96      0.99      0.97        86
            RateBook       1.00      1.00      1.00        80
  SearchCreativeWork       0.93      0.94      0.94       107
SearchScreeningEvent       0.99      0.93      0.96       107

            accuracy                           0.98       700
           macro avg       0.98      0.98      0.98       700
        weighted avg       0.98      0.98      0.98       700



In [ ]:
pd.DataFrame(res)

,ent_type,partial,strict,exact
correct,1658.000000,1600.000000,1541.000000,1600.000000
incorrect,103.000000,0.000000,220.000000,161.000000
partial,0.000000,161.000000,0.000000,0.000000
missed,29.000000,29.000000,29.000000,29.000000
spurious,163.000000,163.000000,163.000000,163.000000
possible,1790.000000,1790.000000,1790.000000,1790.000000
actual,1924.000000,1924.000000,1924.000000,1924.000000
precision,0.861746,0.873441,0.800936,0.831601
recall,0.926257,0.938827,0.860894,0.893855
f1,0.892838,0.904954,0.829833,0.861605


# Load model

Untuk mencoba menjalankan model, diharuskan untuk menjalankan bagian *Installs*, *Imports*, *Tokenize*, dan *Mapping slots and intents*.

In [ ]:
def nlu(text, tokenizer, model, intent_names, slot_names):
    # inputs = tf.constant(tokenizer.encode(text, create_O_slot(text)))[None, :]  # batch_size = 1
    # inputs = tf.constant(tokenizer.encode(text))[None, :]
    encoded_texts = tokenizer(text, padding= "max_length",truncation=True, max_length = max_token_length, return_tensors="tf")
    x = {"input_ids": encoded_texts["input_ids"], "token_type_ids": encoded_texts["token_type_ids"],  "attention_mask": encoded_texts["attention_mask"]}
    inputs = x
    outputs = model(inputs)
    slot_logits, intent_logits = outputs

    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, :]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    info = {"intent": intent_names[intent_id], "slots": {}}
    out_dict = {}
    # get all slot names and add to out_dict as keys
    predicted_slots = set([slot_names[s] for s in slot_ids if s != 0])
    for ps in predicted_slots:
      out_dict[ps] = []

    # check if the text starts with a small letter
    if text[0].islower():
      tokens = tokenizer.tokenize(text, add_special_tokens=True)
    else:
      tokens = tokenizer.tokenize(text)
    slot_list = []
    for token, slot_id in zip(tokens, slot_ids):
        slot_name = slot_names[slot_id]
        if slot_name == "<PAD>":
            continue
        if token == "[CLS]":
            continue
        if token == "[SEP]":
            continue

        # collect tokens
        collected_tokens = [token]
        idx = tokens.index(token)

        # see if it starts with ##
        # then it belongs to the previous token
        if token.startswith("##"):
          # check if the token already exists or not
          if tokens[idx - 1] not in out_dict[slot_name]:
            collected_tokens.insert(0, tokens[idx - 1])
        else:
          slot_list.append(slot_name)
        # add collected tokens to slots
        out_dict[slot_name].extend(collected_tokens)

    # process out_dict
    for slot_name in out_dict:
        tokens = out_dict[slot_name]
        slot_value = tokenizer.convert_tokens_to_string(tokens)

        info["slots"][slot_name] = slot_value.strip()
    info["slot list"] = slot_list

    return info


Sesuaikan path di bawah ini dengan path menuju folder yang sesuai (harus ke `second_pass_epoch_3_weight`). Silakan merujuk pada link https://drive.google.com/drive/folders/1vuf4IZYyoInyYdise5S9LZE9003Lb6sN untuk struktur folder.

In [ ]:
joint_model = tf.keras.models.load_model("path/to/Joint Bert/Snips/second_pass_epoch_3_weight", compile=True)

In [ ]:
nlu("play pray by tokoyami towa and amane kanata", tokenizer, joint_model, intent_names, slot_names)

{'intent': 'PlayMusic',
 'slot list': ['O',
  'B-track',
  'O',
  'B-artist',
  'I-artist',
  'O',
  'B-artist',
  'I-artist'],
 'slots': {'B-artist': 'tokoyami amane',
  'B-track': 'pray',
  'I-album': '',
  'I-artist': 'towa kanata',
  'O': 'play by and'}}

In [ ]:
pred_slot_second_pass = []
pred_intent_second_pass = []
for i in df_snips_test.values:
  info = nlu(i[0], tokenizer, joint_model, intent_names, slot_names)
  pred_slot_second_pass.append(info["slot list"])
  pred_intent_second_pass.append(info["intent"])

In [ ]:
pred_slot_second_pass[:5]

[['O', 'B-entity_name', 'I-artist', 'O', 'O', 'B-playlist', 'I-playlist', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'B-party_size_number',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-spatial_relation',
  'O',
  'B-poi',
  'O',
  'O',
  'B-served_dish',
  'O'],
 ['O',
  'B-artist',
  'I-artist',
  'O',
  'B-playlist_owner',
  'B-playlist',
  'I-playlist',
  'I-playlist',
  'B-playlist',
  'I-playlist',
  'I-playlist',
  'I-playlist',
  'O'],
 ['O',
  'O',
  'B-condition_description',
  'O',
  'B-state',
  'O',
  'B-timeRange',
  'I-timeRange',
  'I-timeRange'],
 ['O', 'B-artist', 'I-artist', 'B-music_item', 'O', 'O', 'O', 'B-sort']]

In [ ]:
pred_intent_second_pass[:5]

['AddToPlaylist', 'BookRestaurant', 'AddToPlaylist', 'GetWeather', 'PlayMusic']